In [32]:
# loading libraries

import pandas as pd
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

## Cargo csvs de features

In [2]:
features_gaby = pd.read_csv('../features/features_gaby.csv')
#features_gonza = pd.read_csv('../features/most_viewed_sku_per_person.csv').drop(columns='Unnamed: 0')
features_ari = pd.read_csv('../features/features_ariel.csv')
#features_mati = pd.read_csv('../features/events_qty_per_person.csv')
labels_training_set = pd.read_csv('../dataset/labels_training_set.csv')
test_set = pd.read_csv('../dataset/trocafone_kaggle_test.csv')

In [3]:
#model_df = pd.merge(features_gaby, features_gonza, how='inner', on='person')
model_df = pd.merge(features_gaby, features_ari, how='inner', on='person')
#model_df = pd.merge(model_df, features_gaby, how='inner', on='person')
model_df.head()

,person,marca_favorita_apple,marca_favorita_asus,marca_favorita_lenovo,marca_favorita_lg,marca_favorita_motorola,marca_favorita_no tiene,marca_favorita_quantum,marca_favorita_samsung,marca_favorita_sony,...,cant_ev_madrugada,cant_ev_tarde,cant_ev_manana,cant_ev_noche,franja_horaria_moda,cant_muy_buenos,cant_excelentes,cant_nuevos,cant_buenos,condicion_de_producto_moda
0,0008ed71,1,0,0,0,0,0,0,0,0,...,0,6,0,0,cant_ev_tarde,1,0,0,2,buenos
1,00091926,1,0,0,0,0,0,0,0,0,...,313,0,0,135,cant_ev_madrugada,163,108,1,102,muy buenos
2,00091a7a,1,0,0,0,0,0,0,0,0,...,0,10,0,0,cant_ev_tarde,0,2,0,1,excelentes
3,000ba417,0,0,0,0,0,0,0,1,0,...,0,149,57,0,cant_ev_tarde,34,11,0,115,buenos
4,000c79fe,1,0,0,0,0,0,0,0,0,...,17,0,0,0,cant_ev_madrugada,0,0,0,4,buenos


## Mergeo el model_df con las labesl de entrenamiento y el modelo con las personas que quiero testear

### Nos quedan dos dataframes, uno de entrenamiento con todos nuestros features y labels
### Y otro tests con todos nuestros features pero sin los labels, los labels es lo que queremos predecir.

In [4]:
training_set = pd.merge(model_df, labels_training_set, how='inner', on='person')
test_set = pd.merge(model_df, test_set, how='inner', on='person')

In [5]:
training_set.drop(columns= 'Unnamed: 0', inplace=True)

In [6]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19414 entries, 0 to 19413
Data columns (total 39 columns):
person                                   19414 non-null object
marca_favorita_apple                     19414 non-null int64
marca_favorita_asus                      19414 non-null int64
marca_favorita_lenovo                    19414 non-null int64
marca_favorita_lg                        19414 non-null int64
marca_favorita_motorola                  19414 non-null int64
marca_favorita_no tiene                  19414 non-null int64
marca_favorita_quantum                   19414 non-null int64
marca_favorita_samsung                   19414 non-null int64
marca_favorita_sony                      19414 non-null int64
nivel_de_actividad_Muy Bajo              19414 non-null int64
nivel_de_actividad_Bajo                  19414 non-null int64
nivel_de_actividad_Medio                 19414 non-null int64
nivel_de_actividad_Alto                  19414 non-null int64
nivel_de_actividad_Muy

## Como se puede observar en nuestro set de entrenamiento predomina el label 0

In [7]:
training_set['label'].value_counts()

0    18434
1      980
Name: label, dtype: int64

### Elimino filas con label igual a 0 para nivelar el dataframe

# Preguntar de cual es la mejor manera de hacer esto

In [8]:
r_training_set = training_set.drop(training_set[training_set.label.eq(0)].sample(13000).index)

In [9]:
r_training_set['label'].value_counts()

0    5434
1     980
Name: label, dtype: int64

In [10]:
r_training_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6414 entries, 2 to 19397
Data columns (total 39 columns):
person                                   6414 non-null object
marca_favorita_apple                     6414 non-null int64
marca_favorita_asus                      6414 non-null int64
marca_favorita_lenovo                    6414 non-null int64
marca_favorita_lg                        6414 non-null int64
marca_favorita_motorola                  6414 non-null int64
marca_favorita_no tiene                  6414 non-null int64
marca_favorita_quantum                   6414 non-null int64
marca_favorita_samsung                   6414 non-null int64
marca_favorita_sony                      6414 non-null int64
nivel_de_actividad_Muy Bajo              6414 non-null int64
nivel_de_actividad_Bajo                  6414 non-null int64
nivel_de_actividad_Medio                 6414 non-null int64
nivel_de_actividad_Alto                  6414 non-null int64
nivel_de_actividad_Muy Alto          

## Elijo que features quiero utilizar

In [33]:
features = ['marca_favorita_apple',
           'marca_favorita_asus',
           'marca_favorita_lenovo',
           'marca_favorita_lg',
           'marca_favorita_motorola',
           'marca_favorita_no tiene',
           'marca_favorita_quantum',
           'marca_favorita_samsung',
           'marca_favorita_sony',
           'nivel_de_actividad_Muy Bajo',
           'nivel_de_actividad_Bajo',
           'nivel_de_actividad_Medio',
           'nivel_de_actividad_Alto',
           'nivel_de_actividad_Muy Alto',
           'mean_events_per_day',
           'cant_total_eventos']

## Creo dos arrays el llamado X es generado a partir de mi dataframe de entrenamiento y los features que quiero tener en cuenta
## Y el segundo Y es generado a partir de sus labels

In [12]:
x = np.array(r_training_set[features])
y = np.array(r_training_set['label'])

## Preguntar si esto es para cross validation

In [23]:
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.10, random_state=7506)
X_train[1]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  6.66666667,
       20.        ])

## Corro KNN para k entre 1 y 203 con paso 2.

In [14]:
best_accuracy = 0
worst_accuracy = 1
best_k = 0
worst_k = 0
best_prediction = []

for i in xrange(1,203,2):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test,pred)
    if (accuracy > best_accuracy) :
        best_k = i
        best_accuracy = accuracy
        best_prediction = pred
    if (accuracy < worst_accuracy) : 
        worst_accuracy = accuracy
        worst_k = i

In [15]:
print 'Best accuracy: ' + str(best_accuracy) + ' with k='  + str(best_k)
print 'Worst accuracy: ' + str(worst_accuracy) + ' with k= ' + str(worst_k)

Best accuracy: 0.8613707165109035 with k=21
Worst accuracy: 0.7881619937694704 with k= 1


## Esto lo hago para ver cuantos 0s y 1s me dio

In [26]:
bunique_elements, counts_elements = np.unique(best_prediction, return_counts=True)
bunique_elements

array([0, 1])

In [27]:
counts_elements

array([639,   3])

## Corro KNN con el mejor K que obtuvimos antes para el set de test

In [28]:
z = np.array(test_set[features])
knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(x,y)
pred = knn.predict(z)

In [29]:
persons = test_set['person']

In [30]:
submit = pd.DataFrame({'label': pred, 'person': persons})
submit = submit[['person', 'label']]

submit.to_csv('../submits/submit_test2_knn.csv', index=False)
submit['label'].value_counts()

0    19306
1      109
Name: label, dtype: int64

In [31]:
primer = pd.read_csv('../submits/1er_submit.csv')
primer['label'].value_counts()

0    18188
1     1227
Name: label, dtype: int64